In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, regexp_extract,sum

In [7]:
spark = SparkSession.builder.appName("DataEngineeringProject").getOrCreate()

In [8]:
df_a = spark.read.csv("P:/softwares and programs/Data engineering projects/Project 1/Dataset/athletes.csv", header=True)
df_c = spark.read.csv("P:/softwares and programs/Data engineering projects/Project 1/Dataset/coaches.csv", header=True)
df_m = spark.read.csv("P:/softwares and programs/Data engineering projects/Project 1/Dataset/medals.csv", header=True)
df_mt = spark.read.csv("P:/softwares and programs/Data engineering projects/Project 1/Dataset/medals_total.csv", header=True)
df_t = spark.read.csv("P:/softwares and programs/Data engineering projects/Project 1/Dataset/technical_officials.csv", header=True)

In [9]:
total_nulls = df_a.agg(*[sum(col(c).isNull().cast("int")).alias(c + "_total_nulls") for c in df_a.columns])
total_nulls.show()

+----------------+----------------------+------------------+----------------------+-----------------------+-------------------------+-------------------+------------------------+----------------------+---------------------------+---------------------------+-----------------------------+-----------------------+---------------+
|name_total_nulls|short_name_total_nulls|gender_total_nulls|birth_date_total_nulls|birth_place_total_nulls|birth_country_total_nulls|country_total_nulls|country_code_total_nulls|discipline_total_nulls|discipline_code_total_nulls|residence_place_total_nulls|residence_country_total_nulls|height_m/ft_total_nulls|url_total_nulls|
+----------------+----------------------+------------------+----------------------+-----------------------+-------------------------+-------------------+------------------------+----------------------+---------------------------+---------------------------+-----------------------------+-----------------------+---------------+
|               

In [10]:
df1_a = df_a.dropna(subset=['gender', 'birth_date', 'discipline'])
df1_a = df1_a.drop('birth_place', 'birth_country', 'residence_place', 'residence_country', 'height_m/ft')

In [11]:
df1_a = df1_a.withColumn("athlete_id", regexp_extract(col("url"), r'n(\d+)-', 1))
df1_a = df1_a.drop('url')

In [12]:
df1_a.show()

+--------------------+-------------+------+----------+--------------------+------------+-------------------+---------------+----------+
|                name|   short_name|gender|birth_date|             country|country_code|         discipline|discipline_code|athlete_id|
+--------------------+-------------+------+----------+--------------------+------------+-------------------+---------------+----------+
|     AALERUD Katrine|    AALERUD K|Female|1994-12-04|              Norway|         NOR|       Cycling Road|            CRD|   1346266|
|         ABAD Nestor|       ABAD N|  Male|1993-03-29|               Spain|         ESP|Artistic Gymnastics|            GAR|   1355250|
|   ABAGNALE Giovanni|   ABAGNALE G|  Male|1995-01-11|               Italy|         ITA|             Rowing|            ROW|   1321381|
|      ABALDE Alberto|     ABALDE A|  Male|1995-12-15|               Spain|         ESP|         Basketball|            BKB|   1322233|
|       ABALDE Tamara|     ABALDE T|Female|1989-

In [13]:
df1_a = df1_a.dropDuplicates(['name'])
df1_a.select([col(c).alias(c + "_null") for c in df1_a.columns]).show()
df1_a.groupBy("athlete_id").count().filter("count > 1").show()

+--------------------+---------------+-----------+---------------+--------------------+-----------------+-------------------+--------------------+---------------+
|           name_null|short_name_null|gender_null|birth_date_null|        country_null|country_code_null|    discipline_null|discipline_code_null|athlete_id_null|
+--------------------+---------------+-----------+---------------+--------------------+-----------------+-------------------+--------------------+---------------+
|     AALERUD Katrine|      AALERUD K|     Female|     1994-12-04|              Norway|              NOR|       Cycling Road|                 CRD|        1346266|
|         ABAD Nestor|         ABAD N|       Male|     1993-03-29|               Spain|              ESP|Artistic Gymnastics|                 GAR|        1355250|
|   ABAGNALE Giovanni|     ABAGNALE G|       Male|     1995-01-11|               Italy|              ITA|             Rowing|                 ROW|        1321381|
|      ABALDE Alberto|

In [15]:
df1_a.count()

11470

In [13]:
total_nulls = df_c.agg(*[sum(col(c).isNull().cast("int")).alias(c + "_total_nulls") for c in df_c.columns])
total_nulls.show()

+----------------+----------------------+------------------+----------------------+------------------------+----------------------+--------------------+-----------------+---------------+
|name_total_nulls|short_name_total_nulls|gender_total_nulls|birth_date_total_nulls|country_code_total_nulls|discipline_total_nulls|function_total_nulls|event_total_nulls|url_total_nulls|
+----------------+----------------------+------------------+----------------------+------------------------+----------------------+--------------------+-----------------+---------------+
|               0|                     0|                 0|                     0|                       0|                     0|                   0|              126|              0|
+----------------+----------------------+------------------+----------------------+------------------------+----------------------+--------------------+-----------------+---------------+



In [14]:
df1_c = df_c.drop('event')
df1_c = df1_c.withColumn("coach_id", regexp_extract(col("url"), r'n(\d+)-', 1))
df1_c = df1_c.drop('url')
df1_c = df1_c.dropDuplicates(['coach_id'])

In [15]:
df1_c.groupBy("coach_id").count().filter("count > 1").show()

+--------+-----+
|coach_id|count|
+--------+-----+
+--------+-----+



In [16]:
df1_c.show()

+--------------------+------------------+------+----------+------------+-----------------+----------+--------+
|                name|        short_name|gender|birth_date|country_code|       discipline|  function|coach_id|
+--------------------+------------------+------+----------+------------+-----------------+----------+--------+
|    SHAHINTAB Mehran|       SHAHINTAB M|  Male|1966-03-21|         IRI|       Basketball|     Coach| 1281792|
|         DANCER Sean|          DANCER S|  Male|1975-08-04|         IRL|           Hockey|Head Coach| 1282059|
|       GRAINGER Adam|        GRAINGER A|  Male|1980-07-06|         IRL|           Hockey|     Coach| 1282063|
|     LORRAINE Andrew|        LORRAINE A|  Male|1972-08-11|         ISR|Baseball/Softball|     Coach| 1282219|
|        MULBERG Nate|         MULBERG N|  Male|1992-01-05|         ISR|Baseball/Softball|     Coach| 1282237|
|         FULI Saiasi|            FULI S|  Male|1978-03-16|         FIJ|     Rugby Sevens|Head Coach| 1282562|
|

In [17]:
df_m = df_m.withColumn("athlete_id", regexp_extract(col("athlete_link"), r'n(\d+)-', 1))
df_m = df_m.drop('athlete_link')

In [18]:
df_m.show()

+------------+----------+--------------------+------------------+-------------------+-----------+------------+---------------+--------------------+--------------------+------------+----------+
|  medal_type|medal_code|          medal_date|athlete_short_name|       athlete_name|athlete_sex|country_code|discipline_code|               event|             country|  discipline|athlete_id|
+------------+----------+--------------------+------------------+-------------------+-----------+------------+---------------+--------------------+--------------------+------------+----------+
|  Gold Medal|         1|2021-07-24 00:00:...|            KIM JD|        KIM Je Deok|          X|         KOR|            ARC|          Mixed Team|   Republic of Korea|     Archery|   1440267|
|  Gold Medal|         1|2021-07-24 00:00:...|              AN S|             AN San|          X|         KOR|            ARC|          Mixed Team|   Republic of Korea|     Archery|   1338248|
|Silver Medal|         2|2021-07-24

In [26]:
df1_m = df_m.join(df1_a.select("athlete_id"), "athlete_id", "inner")
df1_m = df1_m.join(df1_a.select("athlete_id", "gender"), "athlete_id", "inner")
df1_m = df1_m.drop('athlete_sex')
df1_m.show()

+----------+------------+----------+--------------------+------------------+--------------------+------------+---------------+--------------------+--------------------+-------------------+------+
|athlete_id|  medal_type|medal_code|          medal_date|athlete_short_name|        athlete_name|country_code|discipline_code|               event|             country|         discipline|gender|
+----------+------------+----------+--------------------+------------------+--------------------+------------+---------------+--------------------+--------------------+-------------------+------+
|   1350409|  Gold Medal|         1|2021-08-07 00:00:...|           ABALO L|           ABALO Luc|         FRA|            HBL|                 Men|              France|           Handball|  Male|
|   1310126|Silver Medal|         2|2021-07-27 00:00:...|          ABBOTT M|       ABBOTT Monica|         USA|            BSB|            Softball|United States of ...|  Baseball/Softball|Female|
|   1306348|  Gold M

In [20]:
total_nulls = df1_m.agg(*[sum(col(c).isNull().cast("int")).alias(c + "_total_nulls") for c in df1_m.columns])
total_nulls.show()

+----------------------+----------------------+----------------------+----------------------+------------------------------+------------------------+-----------------------+------------------------+---------------------------+-----------------+-------------------+----------------------+------------------+
|athlete_id_total_nulls|medal_type_total_nulls|medal_code_total_nulls|medal_date_total_nulls|athlete_short_name_total_nulls|athlete_name_total_nulls|athlete_sex_total_nulls|country_code_total_nulls|discipline_code_total_nulls|event_total_nulls|country_total_nulls|discipline_total_nulls|gender_total_nulls|
+----------------------+----------------------+----------------------+----------------------+------------------------------+------------------------+-----------------------+------------------------+---------------------------+-----------------+-------------------+----------------------+------------------+
|                     0|                     0|                     0|         

In [21]:
total_nulls = df_mt.agg(*[sum(col(c).isNull().cast("int")).alias(c + "_total_nulls") for c in df_mt.columns])
total_nulls.show()

+----------------+------------------------+----------------------+------------------------+------------------------+-----------------+-------------------+
|Rank_total_nulls|Country Code_total_nulls|Gold Medal_total_nulls|Silver Medal_total_nulls|Bronze Medal_total_nulls|Total_total_nulls|Country_total_nulls|
+----------------+------------------------+----------------------+------------------------+------------------------+-----------------+-------------------+
|               0|                       0|                     0|                       0|                       0|                0|                  0|
+----------------+------------------------+----------------------+------------------------+------------------------+-----------------+-------------------+



In [22]:
total_nulls = df_t.agg(*[sum(col(c).isNull().cast("int")).alias(c + "_total_nulls") for c in df_t.columns])
total_nulls.show()
df_t = df_t.withColumn("to_id", regexp_extract(col("url"), r'n(\d+)-', 1))

+----------------+----------------------+------------------+----------------------+-------------------+----------------------+--------------------+---------------+
|name_total_nulls|short_name_total_nulls|gender_total_nulls|birth_date_total_nulls|country_total_nulls|discipline_total_nulls|function_total_nulls|url_total_nulls|
+----------------+----------------------+------------------+----------------------+-------------------+----------------------+--------------------+---------------+
|               0|                     0|                 0|                     0|                  0|                     0|                   0|              0|
+----------------+----------------------+------------------+----------------------+-------------------+----------------------+--------------------+---------------+



In [23]:
df_t.groupBy('to_id').count().filter("count > 1").show()

+-------+-----+
|  to_id|count|
+-------+-----+
|1573200|    2|
|1573187|    2|
|1573178|    2|
|1573214|    2|
|1573198|    2|
|1572014|    2|
|1573215|    2|
|1572097|    2|
|1573189|    2|
|1573600|    2|
|1572090|    2|
|1573188|    2|
|1572092|    2|
|1573175|    2|
|1572013|    2|
|1571973|    2|
|1573181|    2|
|1572018|    2|
|1571988|    2|
|1573219|    2|
+-------+-----+
only showing top 20 rows



In [24]:
df1_a.toPandas().to_csv('Athletes_updated.csv',index=False)
df1_c.toPandas().to_csv('Coaches_updated.csv',index=False)
df1_m.toPandas().to_csv('Medals_updated.csv',index=False)
df_mt.toPandas().to_csv('Medals_total_updated.csv',index=False)
df_t.toPandas().to_csv('Technical_officials_updated.csv',index=False)


In [181]:
spark.stop()